In [6]:
%conda install pandas

Channels:
 - defaults
Platform: osx-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [84]:
import pandas as pd
import numpy as np

In [87]:
# Reading the CSV files
def read_csv_file(file_path, skiprows=None):
    try:
        df = pd.read_csv(file_path, skiprows=skiprows)
        if 'DATE' in df.columns:  
            df['DATE'] = pd.to_datetime(df['DATE'])
            df.set_index('DATE', inplace=True)
        return df
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# File paths
file_paths = {
    "10-Year Treasury": "/Users/mac/Desktop/FE数据/10-year-Market-Yield.csv",
    "5-Year Treasury": "/Users/mac/Desktop/FE数据/5-year-Market-Yield.csv",
    "1-Year Treasury": "/Users/mac/Desktop/FE数据/1-year-Market-Yield.csv",
    "Baa Corporate Bond Yield": "/Users/mac/Desktop/FE数据/Baa-Moody\'s-Yield.csv",
    "Aaa Corporate Bond Yield": "/Users/mac/Desktop/FE数据/Aaa-Moody\'s-Yield.csv",
    "Core CPI": "/Users/mac/Desktop/FE数据/CORE-CPI.csv",
    "Unemployment Rate": "/Users/mac/Desktop/FE数据/Unemployment-rate.csv",
    "Industrial Production": "/Users/mac/Desktop/FE数据/industrial-production.csv",
    "Momentum_Factor":"/Users/mac/Desktop/FE数据/FF-Momentum-Factor.CSV"
    
}

dataframes = {}
for name, path in file_paths.items():
    df = read_csv_file(path, skiprows=range(14) if name == "Momentum_Factor" else None)
    if df is not None and isinstance(df.index, pd.DatetimeIndex):
        dataframes[name] = df
    else:
        print(f"Index issue with {name}")

# Combining all DataFrames
combined_df = pd.concat([df for df in dataframes.values() if df is not None], axis=1, join='outer')

print(combined_df)

# Resampling to monthly frequency and filling missing values with 0
combined_df_monthly = combined_df.resample('M').ffill().fillna(np.nan)

# Convert the Treasury columns to numeric (float), treating "." as NaN
def convert_to_numeric(df):
    for column in df.columns:
        # Convert the column to numeric, coercing non-numeric values to NaN
        df[column] = pd.to_numeric(df[column].replace('.', np.nan), errors='coerce')
        
    

convert_to_numeric(combined_df_monthly)


# creating new columns
combined_df_monthly['TERM'] = combined_df_monthly['DGS10'] - combined_df_monthly['DGS1']
combined_df_monthly['CURVE'] = combined_df_monthly['DGS10'] - 2*combined_df_monthly['DGS5'] + combined_df_monthly['DGS1']
combined_df_monthly['DEFAULT'] = combined_df_monthly['AAA'] - combined_df_monthly['BAA']
combined_df_monthly['INFLATION'] = np.log(combined_df_monthly['CORESTICKM159SFRBATL']) - np.log(combined_df_monthly['CORESTICKM159SFRBATL'].shift(12))



# Exporting the cleaned DataFrame
output_file_path = "/Users/mac/Desktop/FE数据/combined_financial_data.csv"
combined_df_monthly.to_csv(output_file_path)

# Output file path
print(output_file_path)
combined_df_monthly

Index issue with Momentum_Factor
           DGS10  DGS5  DGS1   BAA   AAA  CORESTICKM159SFRBATL  UNRATE  INDPRO
DATE                                                                          
1919-01-01   NaN   NaN   NaN  7.12  5.35                   NaN     NaN  4.8665
1919-02-01   NaN   NaN   NaN  7.20  5.35                   NaN     NaN  4.6514
1919-03-01   NaN   NaN   NaN  7.15  5.39                   NaN     NaN  4.5170
1919-04-01   NaN   NaN   NaN  7.23  5.44                   NaN     NaN  4.5976
1919-05-01   NaN   NaN   NaN  7.09  5.39                   NaN     NaN  4.6245
...          ...   ...   ...   ...   ...                   ...     ...     ...
2023-12-22  3.90  3.87  4.82   NaN   NaN                   NaN     NaN     NaN
2023-12-25     .     .     .   NaN   NaN                   NaN     NaN     NaN
2023-12-26  3.89  3.89  4.83   NaN   NaN                   NaN     NaN     NaN
2023-12-27  3.79  3.78  4.79   NaN   NaN                   NaN     NaN     NaN
2023-12-28  3.84  3

,DGS10,DGS5,DGS1,BAA,AAA,CORESTICKM159SFRBATL,UNRATE,INDPRO,TERM,CURVE,DEFAULT,INFLATION
DATE,,,,,,,,,,,,
1919-01-31,NaN,NaN,NaN,7.12,5.35,NaN,NaN,4.8665,NaN,NaN,-1.77,NaN
1919-02-28,NaN,NaN,NaN,7.20,5.35,NaN,NaN,4.6514,NaN,NaN,-1.85,NaN
1919-03-31,NaN,NaN,NaN,7.15,5.39,NaN,NaN,4.5170,NaN,NaN,-1.76,NaN
1919-04-30,NaN,NaN,NaN,7.23,5.44,NaN,NaN,4.5976,NaN,NaN,-1.79,NaN
1919-05-31,NaN,NaN,NaN,7.09,5.39,NaN,NaN,4.6245,NaN,NaN,-1.70,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-31,4.09,4.23,5.37,NaN,NaN,NaN,NaN,NaN,-1.28,1.00,NaN,NaN
2023-09-30,4.59,4.60,5.46,NaN,NaN,NaN,NaN,NaN,-0.87,0.85,NaN,NaN
2023-10-31,4.88,4.82,5.44,NaN,NaN,NaN,NaN,NaN,-0.56,0.68,NaN,NaN
